<a href="https://colab.research.google.com/github/tylerlum/ufc_automated_scoring_system/blob/main/UFC_Automated_Scoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFC Data Scraping

The goal of this notebook is to:
* Explore the FightMetrics webpage to scrape the fight and fighter information we need
* Preprocess the data
* Store the fight and fighter data into csv files

Functional as of April 2021

In [1]:
# NUM_EVENTS_INPUT = "All"  #@param {type:"string"}
NUM_EVENTS_INPUT = "2"  #@param {type:"string"}
DATA_MODE_INPUT = "Round by Round"  #@param {type:"string"} #changed from Summary

In [2]:
NUM_EVENTS = None if NUM_EVENTS_INPUT == "All" else int(NUM_EVENTS_INPUT)
ROUND_BY_ROUND = (DATA_MODE_INPUT == "Round by Round")

## Get information about all fighters

In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
from string import ascii_lowercase
from bs4 import BeautifulSoup
import requests

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Set parameters for dataset creation

NUM_EVENTS_INPUT: Integer number of UFC events to get fights from or "All" for all events. There are about 10 fights per event.

DATA_MODE_INPUT: Either "Summary" or "Round by Round". Either get data with columns that are summaries of the whole fight, or summaries round-by-round (more columns).

In [4]:
def get_all_fighters():
    '''Get pandas table of all UFC fighters (Name, Height, Weight, Reach, Record, etc.)'''
    all_fighters_tables = []
    for c in tqdm(ascii_lowercase):
        all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
        all_fighters_table = pd.read_html(all_fighters_url)[0]
        all_fighters_tables.append(all_fighters_table)

    all_fighters = pd.concat(all_fighters_tables)
    return all_fighters

In [5]:
ALL_FIGHTERS = get_all_fighters()
ALL_FIGHTERS.head()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:47<00:00,  1.81s/it]


,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28.0,4.0,0.0,NaN
4,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN


In [6]:
ALL_FIGHTERS.dtypes

First        object
Last         object
Nickname     object
Ht.          object
Wt.          object
Reach        object
Stance       object
W           float64
L           float64
D           float64
Belt        float64
dtype: object

## Clean fighter data

TODO: Convert height, weight, reach to floats.

In [7]:
ALL_FIGHTERS = ALL_FIGHTERS.replace("^-+", np.nan, regex=True)  # Replace -- and --- with nan
ALL_FIGHTERS.dropna(subset=["First", "Last"], how='all')  # Remove rows with no name
ALL_FIGHTERS.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,NaN,155 lbs.,NaN,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,NaN,Orthodox,4.0,6.0,0.0,NaN
3,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28.0,4.0,0.0,NaN
4,David,Abbott,Tank,"6' 0""",265 lbs.,NaN,Switch,10.0,15.0,0.0,NaN


## Helper functions

In [8]:
def get_fighters(fighters_string):
    '''Parses string containing two fighter names. Uses ALL_FIGHTERS global to remove ambiguity in parsing. Returns each fighter name
       Eg. "Robert Whittaker Kelvin Gastelum" => ("Robert Whittaker", "Kelvin Gastelum")'''
    for i, row in ALL_FIGHTERS.iterrows():
        fighter_name = f'{row["First"]} {row["Last"]}'
        if fighters_string.startswith(fighter_name):
            first_fighter = fighter_name
            second_fighter = fighters_string[len(fighter_name)+1:]
            break
    return first_fighter, second_fighter

def remove_duplicates_keep_order(list_):
    '''Removes duplicates while keeping same order'''
    return list(dict.fromkeys(list_))

## Get a list of all UFC events

In [9]:
from urllib.request import urlopen
from string import ascii_uppercase
from dateutil import parser
from datetime import datetime

In [10]:
ALL_PAST_EVENTS_URL = "http://ufcstats.com/statistics/events/completed?page=all"

In [11]:
def get_all_events(all_past_events_url):
    '''Takes in URL to all past events. Returns list of urls, each one representing a UFC event'''
    all_past_events_html = urlopen(all_past_events_url).read().decode("utf-8")
    
    # Regex for "http://ufcstats.com/events-details/<alphanumeric>"
    # Eg. "http://ufcstats.com/event-details/27541033b97c076d"
    pattern = "\"http://ufcstats.com/event-details/[a-zA-Z0-9_]+\""
    all_urls = re.findall(pattern, all_past_events_html)

    # Remove quotes and duplicates
    all_urls = [url.strip("\"") for url in all_urls]
    all_urls = remove_duplicates_keep_order(all_urls)
    return all_urls

In [12]:
# Events
ALL_EVENT_URLS = get_all_events(ALL_PAST_EVENTS_URL)
print(f"Got {len(ALL_EVENT_URLS)} events")
print()

print("Removing the most recent event, since it might not have happened yet")
#ALL_EVENT_URLS = ALL_EVENT_URLS[1:] CHANGE ME BACK LATER!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ALL_EVENT_URLS = ALL_EVENT_URLS[1:]
print(f"Now got {len(ALL_EVENT_URLS)} events")
print(ALL_EVENT_URLS)

Got 653 events

Removing the most recent event, since it might not have happened yet
Now got 652 events
['http://ufcstats.com/event-details/e9e1acc96536bb4f', 'http://ufcstats.com/event-details/a780d16cf7eed44d', 'http://ufcstats.com/event-details/b9415726dc3ec526', 'http://ufcstats.com/event-details/b6c6d1731ff00eeb', 'http://ufcstats.com/event-details/7abe471b61725980', 'http://ufcstats.com/event-details/6f812143641ceff8', 'http://ufcstats.com/event-details/901cddcbfa079097', 'http://ufcstats.com/event-details/3c6976f8182d9527', 'http://ufcstats.com/event-details/51b1e2fd9872005b', 'http://ufcstats.com/event-details/6fb1ba67bef41b37', 'http://ufcstats.com/event-details/15b1b21cd743d652', 'http://ufcstats.com/event-details/3dc3022232b79c7a', 'http://ufcstats.com/event-details/aec273fcb765330d', 'http://ufcstats.com/event-details/e4bb7e483c4ad318', 'http://ufcstats.com/event-details/35080a7f406f9ab3', 'http://ufcstats.com/event-details/1ccff7f0cfdf85eb', 'http://ufcstats.com/event-deta

## Get a list of all UFC events from MMA Decisions

In [13]:
def get_all_mma_decision_events():
    '''Takes in URL to all past events, goes through each year and returns list if urls, each representing 
        a past event from MMA Decicions from newest to oldest'''
    all_href_event_links = []
    for year in range(2023, 1994, -1):
        all_yearly_events_url = f"http://mmadecisions.com/decisions-by-event/{year}/"
        response = requests.get(all_yearly_events_url)
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.find_all("a", href=lambda href: href and "UFC" in href)
        href_list = [link["href"] for link in links]
        all_href_event_links.extend(href_list)
    #all_href_event_links = list(chain.from_iterable(all_href_event_links))
    return all_href_event_links

## Get a list of UFC fights from MMA Decisions

In [14]:
def get_all_mma_decision_fights_in_event(past_event_url):
    '''Takes in a single URL to a past MMA Decision event.
        return fight_urls'''
    response = requests.get(past_event_url)
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a", href=lambda href: href and "decision/" in href)
    href_list = [link["href"] for link in links]
    return href_list

In [15]:
def get_all_fights_in_mma_decision(all_mma_event_urls, num_events=None):
    '''Takes in list of URLs to past events. Return list: urls representing a UFC fight.
    Set num_events to be the number of events to get fights from. Set to None if want all.'''
    if num_events is None:
        num_events = len(all_mma_event_urls)
    
    all_fight_urls = []
    for i, event_url in enumerate(tqdm(all_mma_event_urls[:num_events])):
        # For each event, get the fight urls and winners
        event_url = "http://mmadecisions.com/" + event_url
        fight_urls = get_all_mma_decision_fights_in_event(event_url)
        all_fight_urls.extend(fight_urls)

    return all_fight_urls

In [16]:
all_event_urls = get_all_mma_decision_events()
all_fights_urls = get_all_fights_in_mma_decision(all_event_urls, NUM_EVENTS)
all_fights_urls

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.64it/s]


['decision/14115/Grant-Dawson-vs-Damir-Ismagulov\r\n',
 'decision/14114/Michael-Morales-vs-Max-Griffin\r\n',
 'decision/14113/Ariane-Lipski-vs-Melissa-Gatto\r\n',
 'decision/14108/Karol-Rosa-vs-Yana-Santos\r\n',
 'decision/14106/Luana-Carolina-vs-Ivana-Petrovic\r\n',
 'decision/14105/Alexander-Romanov-vs-Blagoy-Ivanov\r\n',
 'decision/14104/Ilia-Topuria-vs-Josh-Emmett\r\n',
 'decision/14099/Neil-Magny-vs-Philip-Rowe\r\n',
 'decision/14098/Randy-Brown-vs-Wellington-Turman\r\n',
 'decision/14096/Tabatha-Ricci-vs-Gillian-Robertson\r\n',
 'decision/14095/Josh-Van-vs-Zhalgas-Zhumagulov\r\n',
 'decision/14094/Chepe-Mariscal-vs-Trevor-Peek\r\n',
 'decision/14093/Jack-Jenkins-vs-Jamall-Emmers\r\n',
 'decision/14092/Sedriques-Dumas-vs-Cody-Brundage\r\n']

## Get a list of UFC fights

TODO: Right now only sees if result is win. Else sets winner to None. See if this can be improved.

In [17]:
def get_all_fights_in_event(past_event_url, get_results=False):
    '''Takes in a single URL to a past event.
       If get_results=True, returns fight_urls, winners, methods
       else, return fight_urls'''
    # Regex for "http://ufcstats.com/events-details/<alphanumeric>"
    # Eg. "http://ufcstats.com/fight-details/f67aa0b16e16a9ea"
    past_event_html = urlopen(past_event_url).read().decode("utf-8")
    pattern = "\"http://ufcstats.com/fight-details/[a-zA-Z0-9_]+\""
    fight_urls = re.findall(pattern, past_event_html)

    # Remove quotes and duplicates
    fight_urls = [url.strip("\"") for url in fight_urls]
    fight_urls = remove_duplicates_keep_order(fight_urls)

    # Get the winner and method (dec or KO or sub) of each fight
    past_event_table = pd.read_html(past_event_url)[0]  # Will be length 1 list
    winners, methods = [], []
    for _, row in past_event_table.iterrows():
        # TODO: Improve this processing of result
        result = row["W/L"].split(' ')[0]
        if result == "win":
            winner, _ = get_fighters(row["Fighter"])
        else:
            winner = None
        winners.append(winner)
        methods.append(row["Method"])

    if get_results:
        return fight_urls, winners, methods
    else:
        return fight_urls

In [18]:
def get_all_fights(all_event_urls, num_events=None):
    '''Takes in list of URLs to past events. Returns 3 lists: urls, winners, methods, each representing a UFC fight.
       Set num_events to be the number of events to get fights from. Set to None if want all.'''
    if num_events is None:
        num_events = len(all_event_urls)
    
    all_fight_urls, all_winners, all_methods = [], [], []
    for i, event_url in enumerate(tqdm(all_event_urls[:num_events])):
        # For each event, get the fight urls and winners
        fight_urls, winners, methods = get_all_fights_in_event(event_url, get_results=True)
        all_fight_urls.extend(fight_urls)
        all_winners.extend(winners)
        all_methods.extend(methods)
    return all_fight_urls, all_winners, all_methods

In [19]:
FIGHT_URLS, WINNERS, METHODS = get_all_fights(ALL_EVENT_URLS, num_events=NUM_EVENTS)
print(f"Got {len(FIGHT_URLS)} fights")
print(FIGHT_URLS)
print(WINNERS)
print(METHODS)

assert(len(FIGHT_URLS) == len(WINNERS))
assert(len(FIGHT_URLS) == len(METHODS))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.43s/it]

Got 25 fights
['http://ufcstats.com/fight-details/f557405652165aa3', 'http://ufcstats.com/fight-details/4dc5a41f357a7d85', 'http://ufcstats.com/fight-details/0548a24a9bf8d7a0', 'http://ufcstats.com/fight-details/ea29714dcfb07ce2', 'http://ufcstats.com/fight-details/cd137988da724076', 'http://ufcstats.com/fight-details/288f7fb936c1523c', 'http://ufcstats.com/fight-details/962e74250884e4c1', 'http://ufcstats.com/fight-details/f5a6acc99bc5c634', 'http://ufcstats.com/fight-details/fe370c518cf7873e', 'http://ufcstats.com/fight-details/454265014520a3f8', 'http://ufcstats.com/fight-details/dffc65202afb881d', 'http://ufcstats.com/fight-details/bd2e14ee959f9b77', 'http://ufcstats.com/fight-details/c3ef3cb03edde8bb', 'http://ufcstats.com/fight-details/07cb64236ae7aaea', 'http://ufcstats.com/fight-details/2874ea5d2f783e23', 'http://ufcstats.com/fight-details/582806c33ce6dcf6', 'http://ufcstats.com/fight-details/9124740fe7816d70', 'http://ufcstats.com/fight-details/ced01368259428f5', 'http://ufcst

## Get fight tables


In [20]:
def get_labeled_fight_tables(fight_url):
    '''Convert fight url to dictionary of pandas tables of information.
       Before, gave a list of tables that was hard to understand.
       Now have Totals, Per Round Totals, Significant Strikes, Per Round Significant Strikes'''
    fight_tables = pd.read_html(fight_url)
    
    labeled_fight_tables = {}
    labeled_fight_tables['Totals'] = fight_tables[0]
    labeled_fight_tables['Per Round Totals'] = fight_tables[1]
    labeled_fight_tables['Significant Strikes'] = fight_tables[2]
    labeled_fight_tables['Per Round Significant Strikes'] = fight_tables[3]
    return labeled_fight_tables

In [21]:
RAW_FIGHT_TABLES_LIST = []
for url in tqdm(FIGHT_URLS):
    RAW_FIGHT_TABLES_LIST.append(get_labeled_fight_tables(url))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:33<00:00,  1.36s/it]


In [22]:
RAW_FIGHT_TABLES_LIST[0]['Totals'].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Sean Strickland Abus Magomedov,1 0,81 of 188 47 of 101,43% 46%,81 of 188 47 of 101,0 of 0 1 of 2,--- 50%,0 0,0 0,0:10 0:23


In [23]:
RAW_FIGHT_TABLES_LIST[0]['Per Round Totals'].head()

Fighter      KD            Sig. str. Sig. str. %  \
                          Round 1 Round 1              Round 1     Round 1   
                          Round 2 Round 2              Round 2     Round 2   
0  Sean Strickland Abus Magomedov    0  0   11 of 48  32 of 60    22%  53%   
1  Sean Strickland Abus Magomedov    1  0  70 of 140  15 of 41    50%  36%   

            Total str.            Td %           Sub. att    Rev.        Ctrl  
               Round 1         Round 1            Round 1 Round 1     Round 1  
               Round 2         Round 2 Round 2.1  Round 2 Round 2     Round 2  
0   11 of 48  32 of 60  0 of 0  1 of 2  ---  50%     0  0    0  0  0:00  0:23  
1  70 of 140  15 of 41  0 of 0  0 of 0  ---  ---     0  0    0  0  0:10  0:00

In [24]:
RAW_FIGHT_TABLES_LIST[0]['Significant Strikes'].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,Sean Strickland Abus Magomedov,81 of 188 47 of 101,43% 46%,80 of 186 21 of 68,1 of 2 16 of 23,0 of 0 10 of 10,67 of 168 47 of 101,0 of 0 0 of 0,14 of 20 0 of 0


In [25]:
RAW_FIGHT_TABLES_LIST[0]['Per Round Significant Strikes'].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground,Unnamed: 9_level_0
,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1
,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2
0,Sean Strickland Abus Magomedov,11 of 48 32 of 60,22% 53%,10 of 46 10 of 34,1 of 2 13 of 17,0 of 0 9 of 9,11 of 48 32 of 60,0 of 0 0 of 0,0 of 0 0 of 0,NaN
1,Sean Strickland Abus Magomedov,70 of 140 15 of 41,50% 36%,70 of 140 11 of 34,0 of 0 3 of 6,0 of 0 1 of 1,56 of 120 15 of 41,0 of 0 0 of 0,14 of 20 0 of 0,NaN


In [26]:
display(RAW_FIGHT_TABLES_LIST)


[{'Totals':                           Fighter    KD             Sig. str. Sig. str. %  \
  0  Sean Strickland Abus Magomedov  1  0  81 of 188  47 of 101    43%  46%   
  
               Total str.              Td      Td % Sub. att  Rev.        Ctrl  
  0  81 of 188  47 of 101  0 of 0  1 of 2  ---  50%     0  0  0  0  0:10  0:23  ,
  'Per Round Totals':                           Fighter      KD            Sig. str. Sig. str. %  \
                            Round 1 Round 1              Round 1     Round 1   
                            Round 2 Round 2              Round 2     Round 2   
  0  Sean Strickland Abus Magomedov    0  0   11 of 48  32 of 60    22%  53%   
  1  Sean Strickland Abus Magomedov    1  0  70 of 140  15 of 41    50%  36%   
  
              Total str.            Td %           Sub. att    Rev.        Ctrl  
                 Round 1         Round 1            Round 1 Round 1     Round 1  
                 Round 2         Round 2 Round 2.1  Round 2 Round 2     Round 2

## Clean fight information

Separate each fighter's information into a different column

TODO: Lots of stuff to improve. Smarter use of Totals, round by round, and significant strikes. Can also use non integer information, total attempted strikes (not just landed), fighter information, etc. All of those being ignored right now. Find nice way to parse new information round by round. Handle no winner case better. May need to add ignore_index=True for pd.concat

In [27]:
def parse_string(row_string):
    '''Break string into two parts: one for fighter 0 and one for fighter 1
       Eg. 150 of 284  62 of 209 => (150 of 284, 62 of 209)'''
    if not isinstance(row_string, str):
        return "0", "0"
    string_split = row_string.split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [28]:
def convert_to_int_or_double_if_possible(string):
    '''Convert string to int or double if possible
       If has a percent sign, tries to remove it and continue.'''
    def isfloat(value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    # If input is not string, then return it unchanged
    if not isinstance(string, str):
        return string

    # Remove %
    if "%" in string:
        string = string.strip("%")

    # Convert to int or float
    if isfloat(string) and float(string).is_integer():
        return int(string)
    if isfloat(string):
        return float(string)
    return string

In [29]:
def process_fight(raw_fight_table):
    '''Takes in a raw, one-row pandas fight table. Returns a pandas dataframe representing the fight statistics'''    
    # Break up columns.
    # Eg. "Name" => "Fighter 0 Name", "Fighter 1 Name"
    # "KD" => "Fighter 0 KD", "Fighter 1 KD"
    new_columns = []
    for column in raw_fight_table.columns:
        new_columns.append(f"Fighter 0 {column}")
        new_columns.append(f"Fighter 1 {column}")
    print("#########new column")
    print(new_columns)
    # Go through each row and break up the data into the columns
    new_rows = []
    for i, row in raw_fight_table.iterrows():
        new_row = []
        for column in raw_fight_table.columns:
            # Split string at the center space
            stat1, stat2 = parse_string(row[column])

            # TODO: Update this to capture more information

            # Has "100 of 120" type stat. Just store first number
            if " of " in stat1:
                stat1 = stat1.split(" of ")[0]
            if " of " in stat2:
                stat2 = stat2.split(" of ")[0]

            # Has "2:32" type stat (min:sec). Convert to sec.
            if len(re.findall("^[0-9]+:[0-9]+$", stat1)) > 0:
                min1, sec1 = stat1.split(":")[0], stat1.split(":")[1]
                stat1 = convert_to_int_or_double_if_possible(min1)*60 + convert_to_int_or_double_if_possible(sec1)
            if len(re.findall("^[0-9]+:[0-9]+$", stat2)) > 0:
                min2, sec2 = stat2.split(":")[0], stat2.split(":")[1]
                stat2 = convert_to_int_or_double_if_possible(min2)*60 + convert_to_int_or_double_if_possible(sec2)
            
            # Convert string to float or int if possible
            stat1 = convert_to_int_or_double_if_possible(stat1)
            stat2 = convert_to_int_or_double_if_possible(stat2)

            # Add to row
            new_row.append(stat1)
            new_row.append(stat2)
        new_rows.append(new_row)

    # Bring together into new dataframe, then only store the numerical values
    # TODO: Process better to keep more info, not throw so much away
    df = pd.DataFrame(new_rows, columns=new_columns)

    # Add in names, using smarter parsing
    df = df.drop(columns=['Fighter 0 Fighter', 'Fighter 1 Fighter'])
    fighters_string = raw_fight_table["Fighter"][0]  # Only 1 row table
    fighter0, fighter1 = get_fighters(fighters_string)
    df['Fighter 0 Name'] = fighter0
    df['Fighter 1 Name'] = fighter1
    return df

In [30]:
def process_raw_fight_tables(raw_fight_tables, winner, method, round_by_round=False):
    '''Takes in set of raw fight table (one fight), the name of the fight winner, and the method of winning. Returns a cleaned pandas table.
       Set round_by_round=True to use the round-by-round data. Otherwise, uses full fight stats.'''
    def create_aggregated_fight_table(raw_fight_tables):
        # Aggregate data from multiple tables
        fight_table = process_fight(raw_fight_tables["Totals"])
        fight_table2 = process_fight(raw_fight_tables["Significant Strikes"])
        
        # Rename column names with identical data to match
        fight_table2 = fight_table2.rename(columns={"Fighter 0 Sig. str": "Fighter 0 Sig. str.", "Fighter 1 Sig. str": "Fighter 1 Sig. str."})

        # Bring tables together, then remove duplicates
        fight_table = pd.concat([fight_table, fight_table2], axis=1)
        fight_table = fight_table.loc[:,~fight_table.columns.duplicated()]
        return fight_table

    def create_aggregated_round_by_round_fight_table(raw_fight_tables):
        ##### Aggregate data totals table
        tables = []
        print("RAW TABLE!@#@!$@#$!#@$")
        print(raw_fight_tables)
        for i, row in raw_fight_tables["Per Round Totals"].iterrows():
            # Get df of one round
            df = pd.DataFrame(row)
            print("######################")
            print("before")
            print(df)
            print("after")
            values = list(df[i].to_dict().values())
            cols = list(raw_fight_tables["Totals"].columns)
            print("cols##########")
            print(cols)
            df = pd.DataFrame([values], columns=cols)
            print(df)
            # Update columns with round number
            new_cols = [f"Round {i+1} {c}" if  c != "Fighter" else c for c in cols]
            df.columns = new_cols
            print("#####updated")
            print(df)
            tables.append(process_fight(df))
        # Concatenate round-by-round horizontally, so each row is for 1 fight.
        # Then remove duplicates
        totals_df = pd.concat(tables, axis=1)
        totals_df = totals_df.loc[:,~totals_df.columns.duplicated()]
        print("########final totals_df")
        print(totals_df)

        ##### Aggregate data significant strikes table
        tables = []
        for i, row in raw_fight_tables["Per Round Significant Strikes"].iterrows():
            # Get df of one round
            df = pd.DataFrame(row)
            values = list(df[i].to_dict().values())
            cols = list(raw_fight_tables["Significant Strikes"].columns)
            if len(values) != len(cols):
                values = values[:-1]  # Remove last column values, as shown above, has extra column for no reason
            df = pd.DataFrame([values], columns=cols)

            # Update columns with round number
            new_cols = [f"Round {i+1} {c}" if c != "Fighter" else c for c in cols]
            df.columns = new_cols
            tables.append(process_fight(df))
        # Concatenate round-by-round horizontally, so each row is for 1 fight
        # Then remove duplicates
        sig_strikes_df = pd.concat(tables, axis=1)
        sig_strikes_df = sig_strikes_df.loc[:,~sig_strikes_df.columns.duplicated()]
        
        ##### Bring tables together, then remove duplicates
        fight_table = pd.concat([totals_df, sig_strikes_df], axis=1)
        fight_table = fight_table.loc[:,~fight_table.columns.duplicated()]
        return fight_table


    if round_by_round:
        fight_table = create_aggregated_round_by_round_fight_table(raw_fight_tables)
    else:
        fight_table = create_aggregated_fight_table(raw_fight_tables)

    if fight_table["Fighter 0 Name"][0] == winner:
        label = 0
    elif fight_table["Fighter 1 Name"][0] == winner:
        label = 1
    else:
        print(f'ERROR: fight_table["Fighter 0 Name"]={fight_table["Fighter 0 Name"]}, fight_table["Fighter 1 Name"]={fight_table["Fighter 1 Name"]}, winner={winner}')
        label = -1
    fight_table['Winner'] = label
    fight_table['Method'] = method
    return fight_table

In [31]:
FIGHT_TABLE = []
for i in tqdm(range(len(RAW_FIGHT_TABLES_LIST))):
    FIGHT_TABLE.append(process_raw_fight_tables(RAW_FIGHT_TABLES_LIST[i], WINNERS[i], METHODS[i], round_by_round=ROUND_BY_ROUND)) 
FIGHT_TABLE = pd.concat(FIGHT_TABLE, ignore_index=True)
FIGHT_TABLE = FIGHT_TABLE.replace("^-+", np.nan, regex=True)  # Replace -- and --- with nan

  0%|                                                                                                                                | 0/25 [00:00<?, ?it/s]

RAW TABLE!@#@!$@#$!#@$
{'Totals':                           Fighter    KD             Sig. str. Sig. str. %  \
0  Sean Strickland Abus Magomedov  1  0  81 of 188  47 of 101    43%  46%   

             Total str.              Td      Td % Sub. att  Rev.        Ctrl  
0  81 of 188  47 of 101  0 of 0  1 of 2  ---  50%     0  0  0  0  0:10  0:23  , 'Per Round Totals':                           Fighter      KD            Sig. str. Sig. str. %  \
                          Round 1 Round 1              Round 1     Round 1   
                          Round 2 Round 2              Round 2     Round 2   
0  Sean Strickland Abus Magomedov    0  0   11 of 48  32 of 60    22%  53%   
1  Sean Strickland Abus Magomedov    1  0  70 of 140  15 of 41    50%  36%   

            Total str.            Td %           Sub. att    Rev.        Ctrl  
               Round 1         Round 1            Round 1 Round 1     Round 1  
               Round 2         Round 2 Round 2.1  Round 2 Round 2     Round 2  
0

  4%|████▊                                                                                                                   | 1/25 [00:01<00:26,  1.11s/it]

RAW TABLE!@#@!$@#$!#@$
{'Totals':                         Fighter    KD           Sig. str. Sig. str. %  \
0  Damir Ismagulov Grant Dawson  0  0  13 of 28  30 of 57    46%  52%   

            Total str.              Td      Td % Sub. att  Rev.         Ctrl  
0  35 of 54  84 of 128  0 of 0  3 of 5  ---  60%     0  1  1  0  0:08  12:24  , 'Per Round Totals':                         Fighter      KD          Sig. str. Sig. str. %  \
                        Round 1 Round 1            Round 1     Round 1   
                        Round 2 Round 2            Round 2     Round 2   
                        Round 3 Round 3            Round 3     Round 3   
0  Damir Ismagulov Grant Dawson    0  0   2 of 6  23 of 35    33%  65%   
1  Damir Ismagulov Grant Dawson    0  0  10 of 18  5 of 18    55%  27%   
2  Damir Ismagulov Grant Dawson    0  0     1 of 4  2 of 4    25%  50%   

           Total str.            Td %            Sub. att    Rev.        Ctrl  
              Round 1         Round 1    

  8%|█████████▌                                                                                                              | 2/25 [00:01<00:20,  1.13it/s]

...

########final totals_df
   Fighter 0 Round 1 KD  Fighter 1 Round 1 KD  Fighter 0 Round 1 Sig. str.  \
0                     0                     0                           21   

   Fighter 1 Round 1 Sig. str.  Fighter 0 Round 1 Sig. str. %  \
0                           20                             30   

   Fighter 1 Round 1 Sig. str. %  Fighter 0 Round 1 Total str.  \
0                             50                            21   

   Fighter 1 Round 1 Total str.  Fighter 0 Round 1 Td  Fighter 1 Round 1 Td  \
0                            20                     0                     0   

   Fighter 0 Round 1 Td % Fighter 1 Round 1 Td %  Fighter 0 Round 1 Sub. att  \
0                       0                    ---                           0   

   Fighter 1 Round 1 Sub. att  Fighter 0 Round 1 Rev.  Fighter 1 Round 1 Rev.  \
0                           0                       0                       0   

   Fighter 0 Round 1 Ctrl  Fighter 1 Round 1 Ctrl Fighter 0 Name  \
0   

 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏    | 24/25 [00:18<00:00,  1.07it/s]

RAW TABLE!@#@!$@#$!#@$
{'Totals':                          Fighter    KD          Sig. str. Sig. str. %  \
0  Cody Brundage Sedriques Dumas  0  0  3 of 10  17 of 22    30%  77%   

           Total str.              Td      Td % Sub. att  Rev.         Ctrl  
0  31 of 43  77 of 96  2 of 4  0 of 0  50%  ---     1  0  1  2  1:53  11:08  , 'Per Round Totals':                          Fighter      KD         Sig. str. Sig. str. %  \
                         Round 1 Round 1           Round 1     Round 1   
                         Round 2 Round 2           Round 2     Round 2   
                         Round 3 Round 3           Round 3     Round 3   
0  Cody Brundage Sedriques Dumas    0  0  1 of 1  11 of 14   100%  78%   
1  Cody Brundage Sedriques Dumas    0  0    0 of 6  4 of 4    0%  100%   
2  Cody Brundage Sedriques Dumas    0  0    2 of 3  2 of 4    66%  50%   

           Total str.            Td %            Sub. att    Rev.        Ctrl  
              Round 1         Round 1      

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:18<00:00,  1.34it/s]

#########new column
['Fighter 0 Fighter', 'Fighter 1 Fighter', 'Fighter 0 Round 3 Sig. str', 'Fighter 1 Round 3 Sig. str', 'Fighter 0 Round 3 Sig. str. %', 'Fighter 1 Round 3 Sig. str. %', 'Fighter 0 Round 3 Head', 'Fighter 1 Round 3 Head', 'Fighter 0 Round 3 Body', 'Fighter 1 Round 3 Body', 'Fighter 0 Round 3 Leg', 'Fighter 1 Round 3 Leg', 'Fighter 0 Round 3 Distance', 'Fighter 1 Round 3 Distance', 'Fighter 0 Round 3 Clinch', 'Fighter 1 Round 3 Clinch', 'Fighter 0 Round 3 Ground', 'Fighter 1 Round 3 Ground']


In [32]:
FIGHT_TABLE.head(11)

,Fighter 0 Round 1 KD,Fighter 1 Round 1 KD,Fighter 0 Round 1 Sig. str.,Fighter 1 Round 1 Sig. str.,Fighter 0 Round 1 Sig. str. %,Fighter 1 Round 1 Sig. str. %,Fighter 0 Round 1 Total str.,Fighter 1 Round 1 Total str.,Fighter 0 Round 1 Td,Fighter 1 Round 1 Td,Fighter 0 Round 1 Td %,Fighter 1 Round 1 Td %,Fighter 0 Round 1 Sub. att,Fighter 1 Round 1 Sub. att,Fighter 0 Round 1 Rev.,Fighter 1 Round 1 Rev.,Fighter 0 Round 1 Ctrl,Fighter 1 Round 1 Ctrl,Fighter 0 Name,Fighter 1 Name,Fighter 0 Round 2 KD,Fighter 1 Round 2 KD,Fighter 0 Round 2 Sig. str.,Fighter 1 Round 2 Sig. str.,Fighter 0 Round 2 Sig. str. %,Fighter 1 Round 2 Sig. str. %,Fighter 0 Round 2 Total str.,Fighter 1 Round 2 Total str.,Fighter 0 Round 2 Td,Fighter 1 Round 2 Td,Fighter 0 Round 2 Td %,Fighter 1 Round 2 Td %,Fighter 0 Round 2 Sub. att,Fighter 1 Round 2 Sub. att,Fighter 0 Round 2 Rev.,Fighter 1 Round 2 Rev.,Fighter 0 Round 2 Ctrl,Fighter 1 Round 2 Ctrl,Fighter 0 Round 1 Sig. str,Fighter 1 Round 1 Sig. str,Fighter 0 Round 1 Head,Fighter 1 Round 1 Head,Fighter 0 Round 1 Body,Fighter 1 Round 1 Body,Fighter 0 Round 1 Leg,Fighter 1 Round 1 Leg,Fighter 0 Round 1 Distance,Fighter 1 Round 1 Distance,Fighter 0 Round 1 Clinch,Fighter 1 Round 1 Clinch,Fighter 0 Round 1 Ground,Fighter 1 Round 1 Ground,Fighter 0 Round 2 Sig. str,Fighter 1 Round 2 Sig. str,Fighter 0 Round 2 Head,Fighter 1 Round 2 Head,Fighter 0 Round 2 Body,Fighter 1 Round 2 Body,Fighter 0 Round 2 Leg,Fighter 1 Round 2 Leg,Fighter 0 Round 2 Distance,Fighter 1 Round 2 Distance,Fighter 0 Round 2 Clinch,Fighter 1 Round 2 Clinch,Fighter 0 Round 2 Ground,Fighter 1 Round 2 Ground,Winner,Method,Fighter 0 Round 3 KD,Fighter 1 Round 3 KD,Fighter 0 Round 3 Sig. str.,Fighter 1 Round 3 Sig. str.,Fighter 0 Round 3 Sig. str. %,Fighter 1 Round 3 Sig. str. %,Fighter 0 Round 3 Total str.,Fighter 1 Round 3 Total str.,Fighter 0 Round 3 Td,Fighter 1 Round 3 Td,Fighter 0 Round 3 Td %,Fighter 1 Round 3 Td %,Fighter 0 Round 3 Sub. att,Fighter 1 Round 3 Sub. att,Fighter 0 Round 3 Rev.,Fighter 1 Round 3 Rev.,Fighter 0 Round 3 Ctrl,Fighter 1 Round 3 Ctrl,Fighter 0 Round 3 Sig. str,Fighter 1 Round 3 Sig. str,Fighter 0 Round 3 Head,Fighter 1 Round 3 Head,Fighter 0 Round 3 Body,Fighter 1 Round 3 Body,Fighter 0 Round 3 Leg,Fighter 1 Round 3 Leg,Fighter 0 Round 3 Distance,Fighter 1 Round 3 Distance,Fighter 0 Round 3 Clinch,Fighter 1 Round 3 Clinch,Fighter 0 Round 3 Ground,Fighter 1 Round 3 Ground,Fighter 0 Round 4 KD,Fighter 1 Round 4 KD,Fighter 0 Round 4 Sig. str.,Fighter 1 Round 4 Sig. str.,Fighter 0 Round 4 Sig. str. %,Fighter 1 Round 4 Sig. str. %,Fighter 0 Round 4 Total str.,Fighter 1 Round 4 Total str.,Fighter 0 Round 4 Td,Fighter 1 Round 4 Td,Fighter 0 Round 4 Td %,Fighter 1 Round 4 Td %,Fighter 0 Round 4 Sub. att,Fighter 1 Round 4 Sub. att,Fighter 0 Round 4 Rev.,Fighter 1 Round 4 Rev.,Fighter 0 Round 4 Ctrl,Fighter 1 Round 4 Ctrl,Fighter 0 Round 5 KD,Fighter 1 Round 5 KD,Fighter 0 Round 5 Sig. str.,Fighter 1 Round 5 Sig. str.,Fighter 0 Round 5 Sig. str. %,Fighter 1 Round 5 Sig. str. %,Fighter 0 Round 5 Total str.,Fighter 1 Round 5 Total str.,Fighter 0 Round 5 Td,Fighter 1 Round 5 Td,Fighter 0 Round 5 Td %,Fighter 1 Round 5 Td %,Fighter 0 Round 5 Sub. att,Fighter 1 Round 5 Sub. att,Fighter 0 Round 5 Rev.,Fighter 1 Round 5 Rev.,Fighter 0 Round 5 Ctrl,Fighter 1 Round 5 Ctrl,Fighter 0 Round 4 Sig. str,Fighter 1 Round 4 Sig. str,Fighter 0 Round 4 Head,Fighter 1 Round 4 Head,Fighter 0 Round 4 Body,Fighter 1 Round 4 Body,Fighter 0 Round 4 Leg,Fighter 1 Round 4 Leg,Fighter 0 Round 4 Distance,Fighter 1 Round 4 Distance,Fighter 0 Round 4 Clinch,Fighter 1 Round 4 Clinch,Fighter 0 Round 4 Ground,Fighter 1 Round 4 Ground,Fighter 0 Round 5 Sig. str,Fighter 1 Round 5 Sig. str,Fighter 0 Round 5 Head,Fighter 1 Round 5 Head,Fighter 0 Round 5 Body,Fighter 1 Round 5 Body,Fighter 0 Round 5 Leg,Fighter 1 Round 5 Leg,Fighter 0 Round 5 Distance,Fighter 1 Round 5 Distance,Fighter 0 Round 5 Clinch,Fighter 1 Round 5 Clinch,Fighter 0 Round 5 Ground,Fighter 1 

In [33]:
FIGHT_TABLE.tail()

,Fighter 0 Round 1 KD,Fighter 1 Round 1 KD,Fighter 0 Round 1 Sig. str.,Fighter 1 Round 1 Sig. str.,Fighter 0 Round 1 Sig. str. %,Fighter 1 Round 1 Sig. str. %,Fighter 0 Round 1 Total str.,Fighter 1 Round 1 Total str.,Fighter 0 Round 1 Td,Fighter 1 Round 1 Td,Fighter 0 Round 1 Td %,Fighter 1 Round 1 Td %,Fighter 0 Round 1 Sub. att,Fighter 1 Round 1 Sub. att,Fighter 0 Round 1 Rev.,Fighter 1 Round 1 Rev.,Fighter 0 Round 1 Ctrl,Fighter 1 Round 1 Ctrl,Fighter 0 Name,Fighter 1 Name,Fighter 0 Round 2 KD,Fighter 1 Round 2 KD,Fighter 0 Round 2 Sig. str.,Fighter 1 Round 2 Sig. str.,Fighter 0 Round 2 Sig. str. %,Fighter 1 Round 2 Sig. str. %,Fighter 0 Round 2 Total str.,Fighter 1 Round 2 Total str.,Fighter 0 Round 2 Td,Fighter 1 Round 2 Td,Fighter 0 Round 2 Td %,Fighter 1 Round 2 Td %,Fighter 0 Round 2 Sub. att,Fighter 1 Round 2 Sub. att,Fighter 0 Round 2 Rev.,Fighter 1 Round 2 Rev.,Fighter 0 Round 2 Ctrl,Fighter 1 Round 2 Ctrl,Fighter 0 Round 1 Sig. str,Fighter 1 Round 1 Sig. str,Fighter 0 Round 1 Head,Fighter 1 Round 1 Head,Fighter 0 Round 1 Body,Fighter 1 Round 1 Body,Fighter 0 Round 1 Leg,Fighter 1 Round 1 Leg,Fighter 0 Round 1 Distance,Fighter 1 Round 1 Distance,Fighter 0 Round 1 Clinch,Fighter 1 Round 1 Clinch,Fighter 0 Round 1 Ground,Fighter 1 Round 1 Ground,Fighter 0 Round 2 Sig. str,Fighter 1 Round 2 Sig. str,Fighter 0 Round 2 Head,Fighter 1 Round 2 Head,Fighter 0 Round 2 Body,Fighter 1 Round 2 Body,Fighter 0 Round 2 Leg,Fighter 1 Round 2 Leg,Fighter 0 Round 2 Distance,Fighter 1 Round 2 Distance,Fighter 0 Round 2 Clinch,Fighter 1 Round 2 Clinch,Fighter 0 Round 2 Ground,Fighter 1 Round 2 Ground,Winner,Method,Fighter 0 Round 3 KD,Fighter 1 Round 3 KD,Fighter 0 Round 3 Sig. str.,Fighter 1 Round 3 Sig. str.,Fighter 0 Round 3 Sig. str. %,Fighter 1 Round 3 Sig. str. %,Fighter 0 Round 3 Total str.,Fighter 1 Round 3 Total str.,Fighter 0 Round 3 Td,Fighter 1 Round 3 Td,Fighter 0 Round 3 Td %,Fighter 1 Round 3 Td %,Fighter 0 Round 3 Sub. att,Fighter 1 Round 3 Sub. att,Fighter 0 Round 3 Rev.,Fighter 1 Round 3 Rev.,Fighter 0 Round 3 Ctrl,Fighter 1 Round 3 Ctrl,Fighter 0 Round 3 Sig. str,Fighter 1 Round 3 Sig. str,Fighter 0 Round 3 Head,Fighter 1 Round 3 Head,Fighter 0 Round 3 Body,Fighter 1 Round 3 Body,Fighter 0 Round 3 Leg,Fighter 1 Round 3 Leg,Fighter 0 Round 3 Distance,Fighter 1 Round 3 Distance,Fighter 0 Round 3 Clinch,Fighter 1 Round 3 Clinch,Fighter 0 Round 3 Ground,Fighter 1 Round 3 Ground,Fighter 0 Round 4 KD,Fighter 1 Round 4 KD,Fighter 0 Round 4 Sig. str.,Fighter 1 Round 4 Sig. str.,Fighter 0 Round 4 Sig. str. %,Fighter 1 Round 4 Sig. str. %,Fighter 0 Round 4 Total str.,Fighter 1 Round 4 Total str.,Fighter 0 Round 4 Td,Fighter 1 Round 4 Td,Fighter 0 Round 4 Td %,Fighter 1 Round 4 Td %,Fighter 0 Round 4 Sub. att,Fighter 1 Round 4 Sub. att,Fighter 0 Round 4 Rev.,Fighter 1 Round 4 Rev.,Fighter 0 Round 4 Ctrl,Fighter 1 Round 4 Ctrl,Fighter 0 Round 5 KD,Fighter 1 Round 5 KD,Fighter 0 Round 5 Sig. str.,Fighter 1 Round 5 Sig. str.,Fighter 0 Round 5 Sig. str. %,Fighter 1 Round 5 Sig. str. %,Fighter 0 Round 5 Total str.,Fighter 1 Round 5 Total str.,Fighter 0 Round 5 Td,Fighter 1 Round 5 Td,Fighter 0 Round 5 Td %,Fighter 1 Round 5 Td %,Fighter 0 Round 5 Sub. att,Fighter 1 Round 5 Sub. att,Fighter 0 Round 5 Rev.,Fighter 1 Round 5 Rev.,Fighter 0 Round 5 Ctrl,Fighter 1 Round 5 Ctrl,Fighter 0 Round 4 Sig. str,Fighter 1 Round 4 Sig. str,Fighter 0 Round 4 Head,Fighter 1 Round 4 Head,Fighter 0 Round 4 Body,Fighter 1 Round 4 Body,Fighter 0 Round 4 Leg,Fighter 1 Round 4 Leg,Fighter 0 Round 4 Distance,Fighter 1 Round 4 Distance,Fighter 0 Round 4 Clinch,Fighter 1 Round 4 Clinch,Fighter 0 Round 4 Ground,Fighter 1 Round 4 Ground,Fighter 0 Round 5 Sig. str,Fighter 1 Round 5 Sig. str,Fighter 0 Round 5 Head,Fighter 1 Round 5 Head,Fighter 0 Round 5 Body,Fighter 1 Round 5 Body,Fighter 0 Round 5 Leg,Fighter 1 Round 5 Leg,Fighter 0 Round 5 Distance,Fighter 1 Round 5 Distance,Fighter 0 Round 5 Clinch,Fighter 1 Round 5 Clinch,Fighter 0 Round 5 Ground,Fighter 1 

## Combine fight dataset with scorecard dataset

In [38]:
def get_score_card_information(fight_url, win, fighter_0_name):
    '''Takes in a mma decision fight url and winner, returns a table of the scorecard information'''
    tables = pd.read_html(fight_url)
    labeled_scorecard_tables = {}
    labeled_scorecard_tables['Score Card 1'] = tables[6]
    labeled_scorecard_tables['Score Card 2'] = tables[7]
    labeled_scorecard_tables['Score Card 3'] = tables[8]
    
    full_fight_scorecards_table = []
    #set the corresponding fighter_0 and fighter_1 to match ufc_stats
    winner =""
    loser =""
    if win == 0:
        winner ="0"
        loser = "1"
    elif win == 1:
        winner = "1"
        loser = "0"
    else:
        #match fighter_0 based on the last name in the scorecards
        name = fighter_0_name.split()
        fighter_0_last_name = ' '.join(name[1:])
        sc_column_0_fighter = tables[6].iloc[1].values[1]
        if fighter_0_last_name == sc_column_0_fighter:
            winner = "0"
            loser = "1"
        else:
            winner = "1"
            loser = "0"
    for n in range(1, 4):
        tables = []
        for i, row in labeled_scorecard_tables[f"Score Card {n}"].iterrows():
            if i < 2:
                continue
            # Get df of one round
            df = pd.DataFrame(row)
            # get remove the first row
            values = list(df[i].to_dict().values())
            values.pop(0)
            my_list = [f"SC {n}", f"SC {n}"]
            cols = list(my_list)
            df = pd.DataFrame([values], columns=cols)

            # Update columns with round number
            new_cols = [f"Round {i-1} {c}" if c != "Fighter" else c for c in cols]
            # winner is the first fighter, set the add fighter number to column names
            new_columns = []
            new_columns.append(f"Fighter {winner} {new_cols[0]}")
            new_columns.append(f"Fighter {loser} {new_cols[1]}")

            new_cols = new_columns
            df.columns = new_cols
            tables.append(df)
        new_cols = [f"Fighter {winner} SC {n} Total", f"Fighter {loser} SC {n} Total"]
        tables[-1].columns = new_cols

        totals_df = pd.concat(tables, axis=1)
        totals_df = totals_df.loc[:,~totals_df.columns.duplicated()]
        full_fight_scorecards_table.append(totals_df)
    full_fight_scorecards_table = pd.concat(full_fight_scorecards_table, axis=1)
    full_fight_scorecards_table = full_fight_scorecards_table.loc[:,~full_fight_scorecards_table.columns.duplicated()]
    full_fight_scorecards_table = full_fight_scorecards_table.sort_index(axis=1)
    return full_fight_scorecards_table

In [39]:
fight = FIGHT_TABLE.iloc[[0]]
get_score_card_information("http://mmadecisions.com/decision/13850/Jai-Herbert-vs-Ludovit-Klein", "-1", "aksdj Herbert")

,Fighter 0 Round 1 SC 1,Fighter 0 Round 1 SC 2,Fighter 0 Round 1 SC 3,Fighter 0 Round 2 SC 1,Fighter 0 Round 2 SC 2,Fighter 0 Round 2 SC 3,Fighter 0 Round 3 SC 1,Fighter 0 Round 3 SC 2,Fighter 0 Round 3 SC 3,Fighter 0 SC 1 Total,Fighter 0 SC 2 Total,Fighter 0 SC 3 Total,Fighter 1 Round 1 SC 1,Fighter 1 Round 1 SC 2,Fighter 1 Round 1 SC 3,Fighter 1 Round 2 SC 1,Fighter 1 Round 2 SC 2,Fighter 1 Round 2 SC 3,Fighter 1 Round 3 SC 1,Fighter 1 Round 3 SC 2,Fighter 1 Round 3 SC 3,Fighter 1 SC 1 Total,Fighter 1 SC 2 Total,Fighter 1 SC 3 Total
0,10,10,10,9,10,9,9,9,9,28,29,28,9,9,9,10,9,10,9,9,9,28,27,28


In [ ]:
def combine_tables(fight_table, scorecard_fights_urls):
    url_index = 0
    complete_fight_table = []
    fights = []
    scores = []
    decision_methods = ["U-DEC", "S-DEC", "M-DEC"]
    for index in range(len(fight_table)):
        fight = fight_table.iloc[[index]]
        #if fight['Method'] == "U-DEC" or fight['Method'] == "S-DEC" or fight['Method'] == "M-DEC":
        if fight['Method'].isin(decision_methods).any():
            if fight['Winner'].item() == "-1":
                scorecard_table = get_score_card_information("http://mmadecisions.com/"+scorecard_fights_urls[url_index], fight['Winner'].item(), fight['Fighter 0 Name'])
                url_index = url_index + 1
                print("!!!!!!!!!!!fight info")
                print(fight)
                continue
            else:
                scorecard_table = get_score_card_information("http://mmadecisions.com/"+scorecard_fights_urls[url_index], fight['Winner'].item())
                print("###########################")
                print("information")
                print(scorecard_table)
                print(scorecard_table.shape)
                print(fight)
                print(fight.shape)
                print("url index is ")
                print(url_index)
                fight.reset_index(drop=True, inplace=True)
                scorecard_table.reset_index(drop=True, inplace=True)
                row = pd.concat([fight, scorecard_table], axis=1)
                #print(row.shape)
                display(row)
                #print("colums")
                #print(row.columns)
                #print(row)
                fights.append(row)
                url_index = url_index + 1
    
    #fights_df = pd.DataFrame(fights)
    #scores_df = pd.DataFrame(scores)
    #display(scores_df)
    #display(fights_df)
    fights = [pd.DataFrame(fight) for fight in fights]
    complete_fight_table = pd.concat(fights, ignore_index=True)
    complete_fight_table = complete_fight_table.loc[:,~complete_fight_table.columns.duplicated()]

    return complete_fight_table
            

In [ ]:
print(len(all_fights_urls))
print(len(FIGHT_TABLE))
print(FIGHT_TABLE.shape)
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from IPython.display import display
#scorecard_table = get_score_card_information("http://mmadecisions.com/decision/14044/Alex-Caceres-vs-Daniel-Pineda", "1")
#scorecard_table = pd.concat([scorecard_table], axis=1)
#scorecard_table = scorecard_table.loc[:,~scorecard_table.columns.duplicated()]
#scorecard_table = pd.concat([scorecard_table], ignore_index=True)
#print(scorecard_table.shape)
#fight = FIGHT_TABLE.iloc[[0]]
#print(fight.shape)
#row = pd.concat([fight, scorecard_table], axis=1)
#print(row.shape)
#print(row)
#display(row)
#display(scorecard_table)
#for i in range(len(FIGHT_TABLE)):
#    fight = FIGHT_TABLE.iloc[[i]]
#   row = pd.concat([fight, scorecard_table], axis=1)
#   print("hello")
#   print(fight.shape)
#display(FIGHT_TABLE)
tb = combine_tables(FIGHT_TABLE,all_fights_urls)
display(tb)
FIGHT_TABLE = tb

## Augment dataset by flipping around columns

The system should work the same no matter what order we pass in the fighters. Let fighters be A and B. We want

winner(fighter0=A, fighter1=B) = winner(fighter0=B, fighter1=A)

In [ ]:
def create_flipped_table(table):
    '''Rearranges columns of table so that each fight has two rows. Let fighters be A and B.
       One row has (Fighter 0 = A, Fighter 1 = B). One row has (Fighter 0 = B, Fighter 1 = A)
       Ensure same column order, as column names not looked at when passed to ML model'''

    # Get columns in flipped order, which moves the columns around, but changes column name order too
    flipped_columns = []
    for column in table.columns:
        if "Fighter 0" in column:
            flipped_columns.append(column.replace("Fighter 0", "Fighter 1"))
        elif "Fighter 1" in column:
            flipped_columns.append(column.replace("Fighter 1", "Fighter 0"))
        else:
            flipped_columns.append(column)
    flipped_table = table[flipped_columns]

    # Flips winners around
    if 'Winner' in flipped_table.columns:
         flipped_table['Winner'] = flipped_table['Winner'].replace([0, 1], [1, 0])

    # Change column names back to normal
    flipped_table.columns = table.columns
    return flipped_table

In [ ]:
def add_rows_of_flipped_columns(table):
    flipped_table = create_flipped_table(table)
    new_table = pd.concat([table, flipped_table])
    return new_table

In [ ]:
FULL_FIGHT_TABLE = add_rows_of_flipped_columns(FIGHT_TABLE)

In [ ]:
FULL_FIGHT_TABLE.head()

## Example of augmented data

In [ ]:
FULL_FIGHT_TABLE[(FULL_FIGHT_TABLE['Fighter 0 Name'] == "Robert Whittaker") & (FULL_FIGHT_TABLE['Fighter 1 Name'] == "Kelvin Gastelum")]

In [ ]:
FULL_FIGHT_TABLE[(FULL_FIGHT_TABLE['Fighter 1 Name'] == "Robert Whittaker") & (FULL_FIGHT_TABLE['Fighter 0 Name'] == "Kelvin Gastelum")]

## Additional data cleaning

TODO: See if something better than replacing nan with 0. See if something better for labels than 0 and 1. Could remove fights with no winner, or handle them differently. Could remove fights that don't go to decision by removing based on Method.

In [ ]:
X = FIGHT_TABLE.drop(['Winner', 'Fighter 0 Name', 'Fighter 1 Name', 'Method'], axis=1).fillna(0)
y = FIGHT_TABLE[['Winner']]

In [ ]:
X.head()

In [ ]:
y.head()

## Setup train/validate/test split
Can't blindly use full fight table train/validate/test split, because the augmented data must stay together. If in train we know winner(A, B) = A, then we don't want to have winner(B, A) in the validation/test set.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=0)
X_train, y_train = add_rows_of_flipped_columns(X_train), add_rows_of_flipped_columns(y_train)
X_valid, y_valid = add_rows_of_flipped_columns(X_valid), add_rows_of_flipped_columns(y_valid)
X_test, y_test = add_rows_of_flipped_columns(X_test), add_rows_of_flipped_columns(y_test)

In [ ]:
# Expect equal number of examples in Fighter 0 as Fighter 1
assert(len(y_train[y_train['Winner'] == 0]) == len(y_train[y_train['Winner'] == 1]))
assert(len(y_valid[y_valid['Winner'] == 0]) == len(y_valid[y_valid['Winner'] == 1]))
assert(len(y_test[y_test['Winner'] == 0]) == len(y_test[y_test['Winner'] == 1]))

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
print(f"X_train.shape = {X_train.shape}")
print(f"X_valid.shape = {X_valid.shape}")
print(f"X_test.shape = {X_test.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"y_valid.shape = {y_valid.shape}")
print(f"y_test.shape = {y_test.shape}")

## ML Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Train
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)

# Validate
accuracy_train = clf.score(X_train, y_train)
accuracy_valid = clf.score(X_valid, y_valid)
print(f"accuracy_train = {accuracy_train}")
print(f"accuracy_valid = {accuracy_valid}")

In [ ]:
import matplotlib.pyplot as plt

# Visualize importances
plt.rcParams.update({'font.size': 8})
plt.barh(X_train.columns, clf.feature_importances_)

In [ ]:
# MLP
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
accuracy_train = clf.score(X_train, y_train)
accuracy_valid = clf.score(X_valid, y_valid)
print(f"accuracy_train = {accuracy_train}")
print(f"accuracy_valid = {accuracy_valid}")

In [ ]:
# SVM
from sklearn.svm import SVC

clf = SVC(random_state=1).fit(X_train, y_train)
accuracy_train = clf.score(X_train, y_train)
accuracy_valid = clf.score(X_valid, y_valid)
print(f"accuracy_train = {accuracy_train}")
print(f"accuracy_valid = {accuracy_valid}")

In [ ]:
# FFN
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=X_train.shape[1:]))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_train, y_train)
model.evaluate(X_valid, y_valid)

## Test out model manually

In [ ]:
idx = 6

In [ ]:
X_test.iloc[idx]

In [ ]:
# 0 means fighter 0 won. 1 means fighter 1 won.
y_test.iloc[idx]

In [ ]:
X_test.shape

In [ ]:
X_test.iloc[idx].shape

In [ ]:
model.predict(np.expand_dims(X_test.iloc[idx], 0))

## Save data

Store beginning file parameters.
Use current date and time to save files uniquely.

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")
print("dt_string =", dt_string)	

In [ ]:
parameters_string = f"NUM_EVENTS_{NUM_EVENTS_INPUT}_DATA_MODE_{DATA_MODE_INPUT}"
print("parameters_string =", parameters_string)	

In [ ]:
import pickle
filename1 = f"FULL_FIGHT_TABLE_{parameters_string}_{dt_string}.csv"
filename2 = f"FIGHT_TABLE_{parameters_string}_{dt_string}.csv"
filename3 = f"ALL_FIGHTERS_{parameters_string}_{dt_string}.csv"
filename4 = f"RAW_FIGHT_TABLES_LIST_{parameters_string}_{dt_string}.pkl"
print(f"Saving to {filename1} and {filename2} and {filename3} and {filename4}")
FULL_FIGHT_TABLE.to_csv(filename1, index=False)
FIGHT_TABLE.to_csv(filename2, index=False)
ALL_FIGHTERS.to_csv(filename3, index=False)
with open(filename4, 'wb') as handle:
    pickle.dump(RAW_FIGHT_TABLES_LIST, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
new = pd.read_csv(filename1)

In [ ]:
new

In [ ]:
with open(filename4, 'rb') as pickle_file:
    new2 = pickle.load(pickle_file)

In [ ]:
len(new2[0])

## Experimental: Get detailed fighter information

TODO: Get more detailed information about fighters, so we can change the task to fight prediction using fighter stats only. http://ufcstats.com/statistics/fighters?char=a&page=all has little information compared to http://ufcstats.com/fighter-details/33a331684283900f. Still lots to improve. Better features like strikes per minute. Handling nans better. Handling non win/losses better.

In [ ]:
def get_all_fighters_detailed():
    '''Get pandas table with detailed information about all UFC fighters (KO's, strikes, etc.)'''
    fighter_detailed_tables = []
    
    # For each letter of the alphabet, get the fighters
    for c in tqdm(ascii_lowercase):
        # Each page has a list of fighter detail urls
        all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
        all_fighters_html = urlopen(all_fighters_url).read().decode("utf-8")

        # Regex for "http://ufcstats.com/fighter-details/<alphanumeric>"
        # Eg. "http://ufcstats.com/fighter-details/27541033b97c076d"
        pattern = "\"http://ufcstats.com/fighter-details/[a-zA-Z0-9_]+\""
        urls = re.findall(pattern, all_fighters_html)
        
        # Remove quotes and duplicates
        urls = [url.strip("\"") for url in urls]
        urls = remove_duplicates_keep_order(urls)
        
        # For each fighter detail url, merge together their record information
        # Initially in form "Eddie Alvarez Rafael Dos Anjos", "0 0", "1:10, 0:00"
        # Want just "Eddie Alvarez", "0", "1:10", then convert to numbers
        # Just need to get the first value of each one, then average/sum/aggregate this together
        for url in urls:
            fighter_table = pd.read_html(url)[0].dropna(subset=["Time"], how='all')  # Drop initial row of nans

            # If no fight information, add empty dataframe
            if fighter_table.shape[0] == 0:
                df = pd.DataFrame()
                fighter_detailed_tables.append(df)
                continue
                
            # Preprocess certain values for consistency
            # TODO: Handle this better, perhaps keep more information
            fighter_table = fighter_table.drop(columns=["Method", "Event"])
            fighter_table.loc[~fighter_table['W/L'].isin(['win', 'loss']), 'W/L'] = "-1 -1"
            fighter_table.loc[fighter_table['W/L'] == 'win', 'W/L'] = "1  1"
            fighter_table.loc[fighter_table['W/L'] == 'loss', 'W/L'] = "0  0"
            times = [int(min_) * 60 + int(sec) for min_, sec in fighter_table['Time'].str.split(':')]
            fighter_table['Time'] = [f"{t}  {t}" for t in times]
            
            # Parse each row to remove the other fighter's information
            new_rows = []
            for i, row in fighter_table.iterrows():
                # Get df of one round
                df = pd.DataFrame(row, columns=fighter_table.columns)
                values = [row[col] for col in df.columns]
                df = pd.DataFrame([values], columns=fighter_table.columns)
                df = process_fight(df)
                new_rows.append(df)

            # Put rows together, then only keep Fighter 0, then remove "Fighter 0 "
            totals_df = pd.concat(new_rows)
            totals_df = totals_df.loc[:, totals_df.columns.str.contains('Fighter 0')]
            totals_df.columns = [col.replace("Fighter 0 ", "") for col in totals_df.columns]
            totals_df = totals_df.replace("^-+", np.nan, regex=True)  # Replace -- and --- with nan

            # Summarize fighter in 1 row
            new_columns = []
            new_row = []
            for col in totals_df.columns:
                if col == "Name":
                    new_columns.append(col)
                    new_row.append(totals_df[col].iloc[0])
                else:
                    total_col = f"{col} Total"
                    avg_col = f"{col} Avg"
                    new_columns.extend([total_col, avg_col])
                    total = totals_df[col].sum()
                    avg = totals_df[col].mean()
                    new_row.extend([total, avg])
            totals_df = pd.DataFrame([new_row], columns=new_columns)

            fighter_detailed_tables.append(totals_df) 
            break  # Remove this when ready
    all_fighters = pd.concat(fighter_detailed_tables)
    return all_fighters

In [ ]:
x = get_all_fighters_detailed()

In [ ]:
x.head()